<a href="https://colab.research.google.com/github/flaghunter21/DFS/blob/main/MLB_Tiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finding all Combinations of MLB Tiers Lineups

In [238]:
import pandas as pd
import itertools
from itertools import combinations
import numpy as np

In [239]:
df = pd.read_csv('tiers2.csv')
df.head()

,Player,Inj,Likes,Pos,Tier,Order,Hand,Team,Opp,Time,...,WHIP,Avg,STDV,Ceiling,FC Proj,My Proj,Rdm %,Exp.,Used,Con.
0,Byron Buxton,NaN,NaN,OF,T2,2,R,MIN,vs CLE,6:40 PM,...,0,9.6,10.62,35.97,14.73,14.73,50,100,35.40%,45
1,Austin Riley,NaN,NaN,3B,T3,3,R,ATL,vs SF,6:20 PM,...,0,8.7,8.58,31.12,13.96,13.96,50,100,41.80%,51
2,Dansby Swanson,NaN,NaN,SS,T3,2,R,ATL,vs SF,6:20 PM,...,0,8.6,7.43,28.66,13.80,13.80,50,100,41.40%,57
3,Paul Goldschmidt,NaN,NaN,1B,T2,3,R,STL,@ MIL,7:10 PM,...,0,10.9,8.69,31.52,14.14,14.14,50,100,25.70%,60
4,Rafael Devers,NaN,NaN,3B,T2,2,L,BOS,vs DET,6:10 PM,...,0,10.2,7.72,30.10,14.66,14.66,50,100,33.30%,62


In [240]:
print(df.columns)
df = df.loc[:,['Player', 'Tier','My Proj','Used']]
df['Used'] = df['Used'].str.replace('%','').astype(float)/100
df.sort_values('Tier').head()

Index(['Player', 'Inj', 'Likes', 'Pos', 'Tier', 'Order', 'Hand', 'Team', 'Opp',
       'Time', 'vs. Pitcher', 'Win %', 'Total', 'TeamRuns', 'OppRuns',
       'ERA/AVG', 'wOBA', 'ISO', 'Avg innings', 'K/9', 'WHIP', 'Avg', 'STDV',
       'Ceiling', 'FC Proj', 'My Proj', 'Rdm %', 'Exp.', 'Used', 'Con.'],
      dtype='object')


,Player,Tier,My Proj,Used
5,Ronald Acuna,T1,14.71,0.531
6,Jose Altuve,T1,12.28,0.132
8,Vladimir Guerrero Jr.,T1,12.39,0.138
12,Aaron Judge,T1,12.84,0.199
0,Byron Buxton,T2,14.73,0.354


In [241]:
tier_1 = df[df['Tier'] == 'T1']['Player']
tier_2 = df[df['Tier'] == 'T2']['Player']
tier_3 = df[df['Tier'] == 'T3']['Player']
tier_4 = df[df['Tier'] == 'T4']['Player']
tier_5 = df[df['Tier'] == 'T5']['Player']
tier_6 = df[df['Tier'] == 'T6']['Player']

We now find all possible combinations using `itertools.product`.

In [242]:
combos = list(itertools.product(tier_1, tier_2, tier_3, tier_4, tier_5, tier_6))
print(len(combos))
combos_df = pd.DataFrame(combos, columns = ['T1', 'T2', 'T3', 'T4', 'T5', 'T6'])
combos_df.head()

53760


,T1,T2,T3,T4,T5,T6
0,Ronald Acuna,Byron Buxton,Austin Riley,J.D. Martinez,Carlos Correa,Xander Bogaerts
1,Ronald Acuna,Byron Buxton,Austin Riley,J.D. Martinez,Carlos Correa,Michael Brantley
2,Ronald Acuna,Byron Buxton,Austin Riley,J.D. Martinez,Carlos Correa,Luis Arraez
3,Ronald Acuna,Byron Buxton,Austin Riley,J.D. Martinez,Carlos Correa,Christian Yelich
4,Ronald Acuna,Byron Buxton,Austin Riley,J.D. Martinez,Carlos Correa,Yandy Diaz


There are 53,760 possible lineup combinations for this slate. We can map the projections to each player to get every lineup's projection.

In [243]:
projection_map = dict(zip(df['Player'], df['My Proj']))
ownership_map = dict(zip(df['Player'], df['Used']))
ownership_map

{'Aaron Judge': 0.19899999999999998,
 'Alex Bregman': 0.12,
 'Anthony Rizzo': 0.04,
 'Austin Riley': 0.418,
 'Bo Bichette': 0.063,
 'Byron Buxton': 0.354,
 'Carlos Correa': 0.24100000000000002,
 'Cedric Mullins': 0.032,
 'Christian Yelich': 0.036000000000000004,
 'Corey Seager': 0.073,
 'Dansby Swanson': 0.414,
 'George Springer': 0.09,
 'Giancarlo Stanton': 0.066,
 'J.D. Martinez': 0.37799999999999995,
 'J.T. Realmuto': 0.005,
 'Ji-Man Choi': nan,
 'Jose Altuve': 0.132,
 'Juan Soto': 0.084,
 'Kyle Schwarber': 0.132,
 'Kyle Tucker': 0.015,
 'Luis Arraez': 0.35,
 'Luis Robert': 0.015,
 'Marcus Semien': 0.024,
 'Michael Brantley': 0.11699999999999999,
 'Nolan Arenado': 0.249,
 'Paul Goldschmidt': 0.257,
 'Peter Alonso': 0.14300000000000002,
 'Rafael Devers': 0.33299999999999996,
 'Randy Arozarena': 0.006999999999999999,
 'Rhys Hoskins': 0.102,
 'Ronald Acuna': 0.531,
 'Tommy Edman': 0.28600000000000003,
 'Trevor Story': 0.039,
 'Vladimir Guerrero Jr.': 0.138,
 'Willson Contreras': 0.038,

In [244]:
proj_cols = []
own_cols = []
for i in range(1,7):
  combos_df[f'T{i}_Proj'] = combos_df[f'T{i}'].map(projection_map)
  proj_cols.append(f'T{i}_Proj')
for i in range(1, 7):
  combos_df[f'T{i}_Own'] = combos_df[f'T{i}'].map(ownership_map)
  own_cols.append(f'T{i}_Own')
combos_df['Total Proj'] = combos_df[proj_cols].sum(axis=1)
combos_df['Own Sum'] = combos_df[own_cols].sum(axis=1)
combos_df['Own Product'] = np.round(combos_df['T1_Own'] * combos_df['T2_Own']* combos_df['T3_Own'] * combos_df['T4_Own'] * combos_df['T5_Own'] * combos_df['T6_Own'],6)
combos_df.head()

,T1,T2,T3,T4,T5,T6,T1_Proj,T2_Proj,T3_Proj,T4_Proj,...,T6_Proj,T1_Own,T2_Own,T3_Own,T4_Own,T5_Own,T6_Own,Total Proj,Own Sum,Own Product
0,Ronald Acuna,Byron Buxton,Austin Riley,J.D. Martinez,Carlos Correa,Xander Bogaerts,14.71,14.73,13.96,13.0,...,12.39,0.531,0.354,0.418,0.378,0.241,0.417,81.21,2.339,0.002985
1,Ronald Acuna,Byron Buxton,Austin Riley,J.D. Martinez,Carlos Correa,Michael Brantley,14.71,14.73,13.96,13.0,...,10.37,0.531,0.354,0.418,0.378,0.241,0.117,79.19,2.039,0.000837
2,Ronald Acuna,Byron Buxton,Austin Riley,J.D. Martinez,Carlos Correa,Luis Arraez,14.71,14.73,13.96,13.0,...,11.96,0.531,0.354,0.418,0.378,0.241,0.350,80.78,2.272,0.002505
3,Ronald Acuna,Byron Buxton,Austin Riley,J.D. Martinez,Carlos Correa,Christian Yelich,14.71,14.73,13.96,13.0,...,9.23,0.531,0.354,0.418,0.378,0.241,0.036,78.05,1.958,0.000258
4,Ronald Acuna,Byron Buxton,Austin Riley,J.D. Martinez,Carlos Correa,Yandy Diaz,14.71,14.73,13.96,13.0,...,8.38,0.531,0.354,0.418,0.378,0.241,0.010,77.20,1.932,0.000072


In [245]:
combos_df['Total Proj'].describe()

count    53760.000000
mean        69.007762
std          4.498860
min         52.230000
25%         66.560000
50%         69.510000
75%         72.090000
max         81.610000
Name: Total Proj, dtype: float64

In [246]:
clean_combos = combos_df.loc[:, ['T1','T2','T3','T4','T5','T6','Total Proj','Own Sum', 'Own Product']]
clean_combos.head()

,T1,T2,T3,T4,T5,T6,Total Proj,Own Sum,Own Product
0,Ronald Acuna,Byron Buxton,Austin Riley,J.D. Martinez,Carlos Correa,Xander Bogaerts,81.21,2.339,0.002985
1,Ronald Acuna,Byron Buxton,Austin Riley,J.D. Martinez,Carlos Correa,Michael Brantley,79.19,2.039,0.000837
2,Ronald Acuna,Byron Buxton,Austin Riley,J.D. Martinez,Carlos Correa,Luis Arraez,80.78,2.272,0.002505
3,Ronald Acuna,Byron Buxton,Austin Riley,J.D. Martinez,Carlos Correa,Christian Yelich,78.05,1.958,0.000258
4,Ronald Acuna,Byron Buxton,Austin Riley,J.D. Martinez,Carlos Correa,Yandy Diaz,77.20,1.932,0.000072


The lineup combinations have an average of 69 points. The minimum projection is 52.23 and the maximum projection is 81.61.

In [247]:
combos_df.to_csv('combos.csv', index=False)
clean_combos.to_csv('clean_combos.csv',index=False)